In [2]:
#import ....
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [71]:
# Naive data linkage without blocking

# Installing the textdistance library
# !conda install -c conda-forge textdistance (in terminal)

import textdistance

# Loading CSV into a pandas data frame
df_1 = pd.read_csv('google_small.csv', encoding='ISO-8859-1')
df_2 = pd.read_csv('amazon_small.csv', encoding='ISO-8859-1')

### 1. Detecting duplication

df_duplicated_google = df_1[df_1.duplicated()]
df_duplicated_amazon = df_2[df_2.duplicated()]

df_small_google = df_1
df_small_amazon = df_2

### No duplications were found

In [72]:
### Feature engineering 

### Generate new feature with combination of 3 featueres.

# Check for the type.
type(df_small_amazon.description[0])

# Replace missing values (NaN) with empty strings in the Amazon data frame.
df_small_amazon_fix = df_small_amazon.replace(np.nan, '', regex=True)

# Check for and count missing values in the Amazon data frame.
df_small_amazon_fix.isnull().sum()

# Create a new feature in the Amazon data frame by combining the 'title,' 'description,' and 'manufacturer' columns.
df_small_amazon_fix['combined_feature'] = df_small_amazon_fix.title + df_small_amazon_fix.description + df_small_amazon_fix.manufacturer

# Replace missing values (NaN) with empty strings in the Google data frame.
df_small_google_fix = df_small_google.replace(np.nan, '', regex=True)

# Check for and count missing values in the Google data frame.
df_small_google_fix.isnull().sum()

# Create a new feature in the Google data frame by combining the 'name,' 'description,' and 'manufacturer' columns.
df_small_google_fix['combined_feature'] = df_small_google_fix.name + df_small_google_fix.description + df_small_google_fix.manufacturer


In [73]:

### Using the similarity score on the newly engineered features and price

## Cosine similarity function 
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

In [74]:
# Create a list to store matched items by title or name.
matched_list = pd.DataFrame()

# Layer 1 - string match with title VS name
for i in range(1, len(df_small_amazon.title)):
    similar_score = []
    matched_amazon_combined = []
    matched_google_combined = []

    # Create columns to hold the similar price for the further layer
    matched_amazon_title = []
    matched_google_title = []

    matched_amazon_price = []
    matched_google_price = []

    matched_amazon_id = []
    matched_google_id = []

    for j in range(1, len(df_small_google.name)):
        vector1 = text_to_vector(df_small_amazon_fix.title[i])
        vector2 = text_to_vector(df_small_google_fix.name[j])

        similar_combined = get_cosine(vector1, vector2)

        # Append similarity scores and other attributes to lists
        similar_score.append(similar_combined)
        matched_amazon_combined.append(df_small_amazon_fix.combined_feature[i])
        matched_google_combined.append(df_small_google_fix.combined_feature[j])
        matched_amazon_title.append(df_small_amazon_fix.title[i])
        matched_google_title.append(df_small_google_fix.name[j])
        matched_google_id.append(df_small_google.idGoogleBase[j])
        matched_amazon_id.append(df_small_amazon.idAmazon[i])

        # Adding price attributes to matched titles dataframe
        matched_amazon_price.append(df_small_amazon.price[i])
        matched_google_price.append(df_small_google.price[j])

    # Combine lists into a DataFrame
    zippedList = list(zip(
        matched_amazon_id,
        matched_google_id,
        similar_score,
        matched_amazon_title,
        matched_google_title,
        matched_amazon_price,
        matched_google_price
    ))

    all_match_score = pd.DataFrame(zippedList, columns=[
        'matched_amazon_id',
        'matched_google_id',
        'similarity_score',
        'matched_amazon_combined',
        'matched_google_combined',
        'matched_amazon_price',
        'matched_google_price'
    ])

    sorted_data = all_match_score.sort_values('similarity_score')

    # Find the max similarity for each item in the first list
    top_match = sorted_data[-1:]

    # Attach the matches with the max similarity to the main matched_list
    matched_list = pd.concat([matched_list, top_match])

# Sort the matched_list by similarity score in descending order
sorted_match = matched_list.sort_values('similarity_score', ascending=False)

In [132]:
# Load the truth small CSV
true_match_df = pd.read_csv('amazon_google_truth_small.csv', encoding='ISO-8859-1')

# Create a list of true matches
true_match = (true_match_df['idAmazon'] + true_match_df['idGoogleBase']).tolist()

# Thresholds list to find optimal performance
thresholds = [0.1, 0.2, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.38, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# Iterate through different thresholds
for s in thresholds:
    # Filter predicted matches based on the current threshold
    predict_match_full = sorted_match.loc[sorted_match['similarity_score'] >= s]
    predict_match = (predict_match_full['matched_amazon_id'] + predict_match_full['matched_google_id']).tolist()

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    # Calculate True Positives (TP)
    for i in range(0, len(predict_match)):
        for j in range(0, len(true_match)):
            if predict_match[i] == true_match[j]:
                TP += 1

    # Calculate False Negatives (FN) and False Positives (FP)
    FN = len(true_match) - TP
    FP = len(predict_match) - TP

    # Calculate True Negatives (TN)
    TN = len(predict_match) * len(true_match) - TP - FP - FN

    # Calculate accuracy, precision, and recall
    accuracy = (TP + TN) / (TP + FP + FN + TN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)

    # Print evaluation metrics
    print('Now thresholds is:', s)
    print('Length of predict match is:', len(predict_match))
    print('TP:', TP)
    print('FP:', FP)
    print('FN:', FN)
    print('TN:', TN)
    print('')
    print('Accuracy is:', accuracy)
    print('Precision is:', precision)
    print('Recall is:', recall)
    print('============================================')

Now thresholds is : 0.1
length of predict match is : 145
TP : 121
FP : 24
FN : 9
TN : 18696

Accuracy is : 0.9982493368700265
precision is : 0.8344827586206897
recall is : 0.9307692307692308
Now thresholds is : 0.2
length of predict match is : 144
TP : 121
FP : 23
FN : 9
TN : 18567

Accuracy is : 0.9982905982905983
precision is : 0.8402777777777778
recall is : 0.9307692307692308
Now thresholds is : 0.3
length of predict match is : 141
TP : 121
FP : 20
FN : 9
TN : 18180

Accuracy is : 0.998417894162575
precision is : 0.8581560283687943
recall is : 0.9307692307692308
Now thresholds is : 0.31
length of predict match is : 141
TP : 121
FP : 20
FN : 9
TN : 18180

Accuracy is : 0.998417894162575
precision is : 0.8581560283687943
recall is : 0.9307692307692308
Now thresholds is : 0.32
length of predict match is : 140
TP : 121
FP : 19
FN : 9
TN : 18051

Accuracy is : 0.9984615384615385
precision is : 0.8642857142857143
recall is : 0.9307692307692308
Now thresholds is : 0.33
length of predict ma

This linkage method finds similarity based on the Amazon and Google titles and applies the cosine similarity function. Using the best-matched titles from both datasets and setting a threshold of 0.33 on the cosine similarity score is optimal, resulting in Accuracy: 0.896, Precision: 0.86, and Recall: 0.93.

The cosine similarity function outperforms the editing-based and sequence-based string similarity functions.

Overall, the performance based solely on titles is quite good. It could be further improved by applying cosine similarity to measure descriptions after removing insignificant words in the strings. Additionally, implementing price range elimination to reduce False Positives could be another layer of filtering that may enhance performance.




### ================================================================
 Blocking for effcient data linkage

In [301]:
# Efficient Data Linkage through Blocking

# Impute Google manufacturers using other columns

# Read data from CSV files
df_google = pd.read_csv('google.csv', encoding='ISO-8859-1')
df_amazon = pd.read_csv('amazon.csv', encoding='ISO-8859-1')

# Replace missing values with empty strings
df_amazon = df_amazon.replace(np.nan, '', regex=True)
df_google = df_google.replace(np.nan, '', regex=True)

# Extracting some features from 'name' and 'description' for matching
amazon_name_extract = []
amazon_des_extract = []

for i in range(0, len(df_amazon)):
    # Extract numeric values from the 'title' column
    string = df_amazon['title'][i]
    amazon_des_extract.append([int(s) for s in string.split() if s.isdigit()])

# First blocking by price
# Create 5 price blocks based on the spread of price data
# 5 blocks: 0, >$0 - $25, $>25 - $45, $45 - $135, > $135

df_amazon_block_1 = df_amazon[(df_amazon['price'] == 0) | (df_amazon['price'].isna())]
df_amazon_block_2 = df_amazon[(df_amazon['price'] > 0) & (df_amazon['price'] <= 25)]
df_amazon_block_3 = df_amazon[(df_amazon['price'] > 25) & (df_amazon['price'] <= 45)]
df_amazon_block_4 = df_amazon[(df_amazon['price'] > 45) & (df_amazon['price'] <= 135)]
df_amazon_block_5 = df_amazon[(df_amazon['price'] > 135)]

# F_amazon_block_1 is empty, so 4 blocks left

# Removing the unit and converting to a numeric value (price in GBP)
df_google['price'] = df_google['price'].map(lambda x: x.rstrip(' gbp'))
df_google['price'] = pd.to_numeric(df_google['price'])

df_google_block_1 = df_google[(df_google['price'] == 0) | (df_google['price'].isna())]
df_google_block_2 = df_google[(df_google['price'] > 0) & (df_google['price'] <= 25)]
df_google_block_3 = df_google[(df_google['price'] > 25) & (df_google['price'] <= 45)]
df_google_block_4 = df_google[(df_google['price'] > 45) & (df_google['price'] <= 135)]
df_google_block_5 = df_google[(df_google['price'] > 135)]

In [315]:
# Initialize lists to store matching pairs for each block
block2_matching = []
block3_matching = []
block4_matching = []
block5_matching = []

# Reset the index of the dataframes for efficient iteration
df_amazon_block_2 = df_amazon_block_2.reset_index(drop=True)
df_google_block_2 = df_google_block_2.reset_index(drop=True)

df_amazon_block_3 = df_amazon_block_3.reset_index(drop=True)
df_google_block_3 = df_google_block_3.reset_index(drop=True)

df_amazon_block_4 = df_amazon_block_4.reset_index(drop=True)
df_google_block_4 = df_google_block_4.reset_index(drop=True)

df_amazon_block_5 = df_amazon_block_5.reset_index(drop=True)
df_google_block_5 = df_google_block_5.reset_index(drop=True)

# Define a function for matching and appending to the appropriate block
def match_and_append(amazon_df, google_df, matching_list):
    for i in range(len(amazon_df)):
        for j in range(len(google_df)):
            amazon_id = amazon_df.loc[i, 'idAmazon']
            google_id = google_df.loc[j, 'id']
            matching_list.append((amazon_id, google_id))

# Matching for each block
match_and_append(df_amazon_block_2, df_google_block_2, block2_matching)
match_and_append(df_amazon_block_3, df_google_block_3, block3_matching)
match_and_append(df_amazon_block_4, df_google_block_4, block4_matching)
match_and_append(df_amazon_block_5, df_google_block_5, block5_matching)

# Combine all matching blocks into one list
block_predict_match = block2_matching + block3_matching + block4_matching + block5_matching

In [335]:
block_predict_list[2987]

'b0000ycfcwhttp://www.google.com/base/feeds/snippets/9558419339728905800'

In [332]:
### e.g Extract number from features
str = df_amazon['title'][0]
[int(s) for s in str.split() if s.isdigit()]

[950, 0]

In [144]:
### e.g. Extract string from name and decription 
str = "learning quickbooks 2007"
[int(s) for s in str.split() if s.isdigit()]

[2007]

In [338]:
# Load the ground truth data
ground_truth_match_df = pd.read_csv('amazon_google_truth.csv', encoding='ISO-8859-1')

# Combine two IDs into a string
ground_truth_match = (ground_truth_match_df['idAmazon'] + ground_truth_match_df['idGoogleBase']).tolist()

# Initialize the counting for results
tp = 0
tn = 0
fp = 0 
fn = 0 

for i in range(0, len(block_predict_match)):
    for j in range(0,len(ground_truth_match)):
        if block_predict_match[i] == ground_truth_match[j]:
            tp += 1

In [ ]:
# Impute Google manufacturers by other columns

# Read data from CSV files
df_google = pd.read_csv('google.csv', encoding='ISO-8859-1')
df_amazon = pd.read_csv('amazon.csv', encoding='ISO-8859-1')

# Replace missing values with empty strings
df_amazon = df_amazon.replace(np.nan, '', regex=True)
df_google = df_google.replace(np.nan, '', regex=True)

# Extracting some features from 'name' and 'description' for matching
amazon_name_extract = []
amazon_des_extract = []

for i in range(0, len(df_amazon)):
    # Extract numeric values from the 'title' column
    str = df_amazon['title'][i]
    amazon_des_extract.append([int(s) for s in str.split() if s.isdigit()])

# First blocking by price
# Create 5 price blocks based on the spread of price data
# 5 blocks: 0, >$0 - $25, $>25 -$45, $45 - $135, > $135

df_amazon_block_1 = df_amazon[(df_amazon['price'] == 0) | (df_amazon['price'].isna())]
df_amazon_block_2 = df_amazon[(df_amazon['price'] > 0) & (df_amazon['price'] <= 25)]
df_amazon_block_3 = df_amazon[(df_amazon['price'] > 25) & (df_amazon['price'] <= 45)]
df_amazon_block_4 = df_amazon[(df_amazon['price'] > 45) & (df_amazon['price'] <= 135)]
df_amazon_block_5 = df_amazon[(df_amazon['price'] > 135)]

# F_amazon_block_1 is empty, so 4 Blocks left

# Removing the unit and converting to a numeric value (price in GBP)
df_google['price'] = df_google['price'].map(lambda x: x.rstrip(' gbp'))
df_google['price'] = pd.to_numeric(df_google['price'])

df_google_block_1 = df_google[(df_google['price'] == 0) | (df_google['price'].isna())]
df_google_block_2 = df_google[(df_google['price'] > 0) & (df_google['price'] <= 25)]
df_google_block_3 = df_google[(df_google['price'] > 25) & (df_google['price'] <= 45)]
df_google_block_4 = df_google[(df_google['price'] > 45) & (df_google['price'] <= 135)]
df_google_block_5 = df_google[(df_google['price'] > 135)]

In [ ]:
# Efficient Data Linkage through Blocking

# Impute Google manufacturers using other columns

# Read data from CSV files
df_google = pd.read_csv('google.csv', encoding='ISO-8859-1')
df_amazon = pd.read_csv('amazon.csv', encoding='ISO-8859-1')

# Replace missing values with empty strings
df_amazon = df_amazon.replace(np.nan, '', regex=True)
df_google = df_google.replace(np.nan, '', regex=True)

# Extracting some features from 'name' and 'description' for matching
amazon_name_extract = []
amazon_des_extract = []

for i in range(0, len(df_amazon)):
    # Extract numeric values from the 'title' column
    string = df_amazon['title'][i]
    amazon_des_extract.append([int(s) for s in string.split() if s.isdigit()])

# First blocking by price
# Create 5 price blocks based on the spread of price data
# 5 blocks: 0, >$0 - $25, $>25 - $45, $45 - $135, > $135

df_amazon_block_1 = df_amazon[(df_amazon['price'] == 0) | (df_amazon['price'].isna())]
df_amazon_block_2 = df_amazon[(df_amazon['price'] > 0) & (df_amazon['price'] <= 25)]
df_amazon_block_3 = df_amazon[(df_amazon['price'] > 25) & (df_amazon['price'] <= 45)]
df_amazon_block_4 = df_amazon[(df_amazon['price'] > 45) & (df_amazon['price'] <= 135)]
df_amazon_block_5 = df_amazon[(df_amazon['price'] > 135)]

# F_amazon_block_1 is empty, so 4 blocks left

# Removing the unit and converting to a numeric value (price in GBP)
df_google['price'] = df_google['price'].map(lambda x: x.rstrip(' gbp'))
df_google['price'] = pd.to_numeric(df_google['price'])

df_google_block_1 = df_google[(df_google['price'] == 0) | (df_google['price'].isna())]
df_google_block_2 = df_google[(df_google['price'] > 0) & (df_google['price'] <= 25)]
df_google_block_3 = df_google[(df_google['price'] > 25) & (df_google['price'] <= 45)]
df_google_block_4 = df_google[(df_google['price'] > 45) & (df_google['price'] <= 135)]
df_google_block_5 = df_google[(df_google['price'] > 135)]

In [ ]:
# Initialize lists to store matching pairs for each block
block2_matching = []
block3_matching = []
block4_matching = []
block5_matching = []

# Reset the index of the dataframes for efficient iteration
df_amazon_block_2 = df_amazon_block_2.reset_index(drop=True)
df_google_block_2 = df_google_block_2.reset_index(drop=True)

df_amazon_block_3 = df_amazon_block_3.reset_index(drop=True)
df_google_block_3 = df_google_block_3.reset_index(drop=True)

df_amazon_block_4 = df_amazon_block_4.reset_index(drop=True)
df_google_block_4 = df_google_block_4.reset_index(drop=True)

df_amazon_block_5 = df_amazon_block_5.reset_index(drop=True)
df_google_block_5 = df_google_block_5.reset_index(drop=True)

# Define a function for matching and appending to the appropriate block
def match_and_append(amazon_df, google_df, matching_list):
    for i in range(len(amazon_df)):
        for j in range(len(google_df)):
            amazon_id = amazon_df.loc[i, 'idAmazon']
            google_id = google_df.loc[j, 'id']
            matching_list.append((amazon_id, google_id))

# Matching for each block
match_and_append(df_amazon_block_2, df_google_block_2, block2_matching)
match_and_append(df_amazon_block_3, df_google_block_3, block3_matching)
match_and_append(df_amazon_block_4, df_google_block_4, block4_matching)
match_and_append(df_amazon_block_5, df_google_block_5, block5_matching)

# Combine all matching blocks into one list
block_predict_match = block2_matching + block3_matching + block4_matching + block5_matching

In [345]:
# Load the ground truth data
ground_truth_match_df = pd.read_csv('amazon_google_truth.csv', encoding='ISO-8859-1')

# Combine two IDs into a string
ground_truth_match = (ground_truth_match_df['idAmazon'] + ground_truth_match_df['idGoogleBase']).tolist()

# Initialize the counting for results
tp = 0
tn = 0
fp = 0 
fn = 0 

for i in range(0, len(block_predict_match)):
    for j in range(0,len(ground_truth_match)):
        if block_predict_match[i] == ground_truth_match[j]:
            tp += 1

# Calculate false negatives (FN)
fn = (len(ground_true_match)) - tp

# Calculate false positives (FP)
fp = len(block_predict_match) - tp

# Calculate true negatives (TN)
tn = len(block_predict_match) * len(ground_true_match) - tp - fp - fn

# Calculate the total number of comparisons (N)
n = len(block_predict_match) * len(ground_true_match)

# Calculate Pair Completeness (PC)
PC = tp / (tp + fn)

# Calculate Reduction Ratio (RR)
RR = 1 - ((tp + fp) / n)

# Print the results
print('Length of predict match is:', len(block_predict_match))
print('TP:', tp)
print('FP:', fp)
print('FN:', fn)
print('TN:', tn)
print('')
print('Pair completeness PC is:', PC)
print('Reduction ratio RR is:', RR)

print('============================================')

length of predict match is : 3
TP : 982
FR : 962696
FN : 318
TN : 1251817404

Pair completeness PC is : 0.7553846153846154
Reduction ratio RR is : 0.9992307692307693


Comments:

Choice of blocking method: The first blocking method involves dividing the data into 5 blocks based on the spread of price data: | 0 | >0 - 25 | >25 - 45 | 45 - 135 | > 135 |. This choice is informed by the quartiles and median of both Amazon and Google datasets.

In the second stage, elements within each block are reorganized by extracting strings from the name and description fields.

Methods related to the two quality measures:

The blocking methods involve dividing the data by price into 4 blocks for each dataset. This reduction in data volume helps improve computational efficiency for data linkage.
Pair Completeness (PC) is calculated as follows: PC = tp / (tp + fn). An increase in the proportion of actual positives matches correctly identified (TP) leads to an increase in Pair Completeness, indicating effective blocking.

Reduction Ratio (RR) is calculated as: RR = 1 - (tp + fp) / n. A high RR indicates better performance of the blocking method, offering greater efficiency for record linkage between datasets.

### ==============================================================